In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import reciprocalspaceship as rs
import requests

# This cell defines several helpful functions for dealing with complex structure factors and Argand diagrams
def to_amplitude_and_phase(cplx, deg=True):
    """ Convert complex numbers, cplx, to amplitudes and phases"""
    amplitude = np.abs(cplx)
    phase = np.angle(cplx)
    if deg:
        phase = np.rad2deg(phase)
    return amplitude, phase

def to_cmplx(amplitude, phase, deg=True):
    """ Convert amplitudes and phases to complex numbers """
    if deg:
        phase = np.deg2rad(phase)
    cplx = amplitude * np.exp(1j*phase)
    return cplx

def arrow(x1, y1, x2, y2, head_scale=0.1, head_width=None, **kwargs):
    """ Plot an arrow from x1,y1 -> x2,y2 """
    dx,dy = x2-x1,y2-y1
    norm = np.sqrt(dx*dx + dy*dy)
    l = head_scale*norm
    w = head_width if head_width is not None else 1.0*l
    plt.arrow(x1, y1, dx, dy, head_width=w, head_length=l, length_includes_head=True, **kwargs)

def argand(cplx1, cplx2, color='k', **kwargs):
    """ Add an arrow pointing from cplx1 to cplx2 in the complex plane """
    x1,y1 = np.real(cplx1),np.imag(cplx1)
    x2,y2 = np.real(cplx2),np.imag(cplx2)
    arrow(x1, y1, x2, y2, ec=color, fc=color, **kwargs)

    #axis labels
    plt.xlabel("$\mathbb{Re}$", fontsize=20)
    plt.ylabel("$\mathbb{Im}$", fontsize=20)
    
    #modify the axis limits to keep everything in the plot
    #no need to understand this
    lim = 1.05*max(np.abs(cplx1), np.abs(cplx2))
    lim = max(plt.gca().get_xlim()[1], lim)
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.gca().set_aspect('equal')
    plt.grid(ls='-.')   

#Define some nice, colorblind-friendly colors
c1,c2,c3 = "#1b9e77", "#d95f02", "#7570b3"



In [ ]:
#Download an mtz File from RCSB
PDBID="1lzt"
mtz_url = f"https://edmaps.rcsb.org/coefficients/{PDBID}.mtz"
mtz_file = f"{PDBID}.mtz"
response = requests.get(mtz_url)
with open(mtz_file, 'wb') as out:
    out.write(response.content)

#Load the mtz file using reciprocalspaceship
#dataset is an rs.DataSet object -- essentially a pandas DataFrame that understand crystallography
#pandas https://pandas.pydata.org/docs
#reciprocalspaceship https://hekstra-lab.github.io/reciprocalspaceship/
dataset = rs.read_mtz(mtz_file)

#Show the first 10 reflections in the dataset
dataset.head(10)

In [ ]:
#Sample two *acentric* structure factors at random
sample = dataset.acentrics.sample(2)

#Use the 
# - experimental structure factor amplitudes (FP column)
# - phases calculated from the atomic model (PHIC column)
F1,F2 = sample.to_structurefactor('FP', 'PHIC')
#^^F1 and F2 are complex numbers

print(f"""
#Amplitude and phase representation
AMPLITUDE1,PHI1 = {np.abs(F1), np.rad2deg(np.angle(F1))}
AMPLITUDE2,PHI2 = {np.abs(F2), np.rad2deg(np.angle(F2))}

#Complex number representation
F1 = {F1}
F2 = {F2}
""")

In [ ]:
plt.figure()
argand(0., F1, color=c1, label='F1')
argand(0., F2, color=c2, label='F2')
lim = max(np.abs(F1), np.abs(F2))
plt.legend()

In [ ]:
#Example: complex conjugates
F1star = np.conjugate(F1)
F2star = np.conjugate(F2)

plt.figure()
argand(0., F1star, color=c1, label='F1*')
argand(0., F2star, color=c2, label='F2*')
plt.legend()

In [ ]:
# Example: Adding structure factors in the complex plane
F3 = F1 + F2 

plt.figure()
argand(0, F1, color=c1, label='F1')
argand(0, F2, color=c2, label='F2')
argand(0, F3, color=c3, label='F1 + F2')
plt.legend()

plt.figure()
argand(0, F1, color=c1, label='F1')
argand(F1, F3, color=c2, label='F2')
argand(0, F3, color=c3, label='F1 + F2')
plt.legend()

In [ ]:
# Example: Rotating structure factors in the complex plane
deltaphi = 70. #degrees
radians = np.deg2rad(deltaphi)
F4 = F1*np.exp(1j*radians)
argand(0, F1, color=c1, label='F1')
argand(0, F4, color=c2, label='F4')
plt.legend()
plt.title(f"The angle between F1 and F4 is: {deltaphi} $^\circ$.")